#Pivot driven pendulum lab by Solomon Daramay
In this notebook we will be looking at the rigid pendulum whose pivot point moves in a circle around the pendulum bob.


##Step 1: Import needed modules

In [ ]:
#import modules
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad

##Step 2: define the animation function
This function was provided in the assignment details. It will allow us to make an animation of the driven pendulum.

In [ ]:
# create pendulum animation
def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    """
    Creates an animation of the pivot-driven pendulum.

    PARAMETERS:
    t : ndarray
        Time array.
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    l : float, optional
        pendulum length (default is 1).
    show_traj : bool, optional
        toggle showing the trajectory of the pendulum bob (default is False).
    speed_factor : float, optional
        factor to scale the animation speed (default is 1.0).
        a value > 1.0 slows down the animation, < 1.0 speeds it up.

    RETURNS:
    anim : FuncAnimation
        matplotlib animation object.
    """
    fig, ax = plt.subplots(figsize=(4, 4)) # Increased figure size
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    # ax.set_aspect('equal', adjustable='box')
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # Initialize plot elements
    pivot, = ax.plot([], [], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([], [], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([], [], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None # Initialize trajectory to None

    if show_traj:
        trajectory, = ax.plot([], [], '-', color='gray', lw=1, alpha=0.5, label='Trajectory') # Add trajectory line

    def animate(i):
        # Update the positions of the plot elements
        pivot.set_data([x_pivot[i]], [y_pivot[i]]) # Pass as sequences
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]]) # Pass as sequences

        artists = [pivot, pendulum_arm, pendulum_bob] # List of artists to update

        if show_traj and trajectory:
             trajectory.set_data(x_lab[:i+1], y_lab[:i+1]) # Update trajectory data
             artists.append(trajectory) # Add trajectory to the list of artists

        return artists # Return all updated artists

    # Create the animation
    # Adjust the interval based on the average time step in t
    # This aims to make the animation speed consistent with the simulation time
    average_time_step = np.mean(np.diff(t))
    # Scale the interval by the speed_factor
    interval = average_time_step * 1000 * speed_factor # Convert to milliseconds and apply speed_factor.

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig) # Close the initial figure to prevent it from displaying

    return anim

##Step 3: Define the differential equation

The differential equation governing the motion of the driven rigid pendulum is given by:

$$ \ddot{\theta} + 2\gamma\dot{\theta} + \frac{g + \ddot{y}_{\text{pivot}}}{l} \sin\theta + \frac{\ddot{x}_{\text{pivot}}}{l} \cos\theta = 0 $$

Where $\theta$ is the angle of the pendulum with respect to the vertical, $\gamma$ is the damping coefficient, $g$ is the acceleration due to gravity, $l$ is the length of the pendulum, and $\ddot{x}_{\text{pivot}}$ and $\ddot{y}_{\text{pivot}}$ are the horizontal and vertical accelerations of the pivot point, respectively.

(I would like to mention that the above text was written by AI, not me)

Now we just have to put this into a function that is usable by solve_ivp(), shown below:

In [ ]:
#define the DE
def driven(t, y, g, l, omega_d, gamma):
  theta, ang_vel = y
  dtheta_dt = ang_vel
  # Accelerations of the pivot
  a_x = l * omega_d**2 * np.sin(omega_d * t)
  a_y = -l * omega_d**2 * np.cos(omega_d * t)
  # Equation of motion for driven pendulum
  d2theta_dt = -2*gamma*ang_vel - (g+a_y)/l * np.sin(theta) - a_x/l * np.cos(theta)
  dy_dt = [dtheta_dt, d2theta_dt]
  return dy_dt

##Step 4: Set initial values and define the coordinate functions
To test that our differential equation and solve_ivp solution works, we can run it with some simple initial values. Everything below is either a 1 or 0. I would like to mention that our initial angular velocity must be proportional in some way to the driving force. Because the initial angular velocity is relative to the pivot, and the pivot is moving, there must be an inital angular velocity for our pendulum bob to be stationary relative to the lab frame.

In [ ]:
# initial values
g = 1 #gravity
theta = 0 #initial angle of bob
omega_0 = 1 #natural frequency
l=1 #length of pivot arm
omega_d = 1 #driving force
ang_vel = omega_d/l #initial angular velocity. Note: needs to be proportional to omega_d in order to be stationary at t=0
gamma = 0 #damping
x_p0 = 0
y_p0 = 0

#packing state variable
y_0 = [theta, ang_vel]

# range of times for integrating
t_min = 0
t_max = 10
t_span = [t_min, t_max] # time span for solve_ivp


##Step 5: Define a few more necessary functions

`get_pivot_xy`: Calculates two lists, an x and y coordinate list of the pivot point throughout the time interval given the initial conditions

`get_pendulum_xy`: Calculates the position of the pendulum bob relative to the pivot

`get_lab_xy`: Calculates the position of the pendulum bob inside our reference frame used in the animation[link text](https://)

In [ ]:
#get pivot
def get_pivot_xy(t_plot, omega_d, x_p0, y_p0):
  x_pivot = -l * np.sin(omega_d * t_plot)
  y_pivot = -l + l * np.cos(omega_d * t_plot)
  return x_pivot, y_pivot

def get_pendulum_xy(t_plot, theta_plot, l):
  x_pendulum = l * np.sin(theta_plot)
  y_pendulum = -l * np.cos(theta_plot)
  return x_pendulum, y_pendulum

def get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum):
  x_lab = x_pivot + x_pendulum
  y_lab = y_pivot + y_pendulum
  return x_lab, y_lab

##Step 6: Use solve_ivp() to find a solution to the differential equation

The best method for this particular case is Radau because of it's high accuracy in numerical systems. I printed the solution here so that it's visible to make sure it looks good.[link text](https://)

In [ ]:
#initial solution testing
sol = solve_ivp(driven, t_span, y_0, args=(g, l, omega_d, gamma),
                    dense_output=True,
                    method='Radau'
                    )
print(sol)

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  9.990e-04 ...  9.934e+00  1.000e+01]
        y: [[ 0.000e+00  9.990e-04 ...  7.667e+00  7.583e+00]
            [ 1.000e+00  1.000e+00 ... -1.226e+00 -1.336e+00]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7be17a6baab0>
 t_events: None
 y_events: None
     nfev: 255
     njev: 4
      nlu: 36


##Step 7: Animate the solution to the driven rigid pendulum
Now all that's left is to animate our solution and see if it works:



In [ ]:
# animate motion
# plot pendulum angle versus time
# get coordinates for plotting
n_points = 300 # number of frames
t_plot = np.linspace(t_min, t_max, n_points)
theta_plot = sol.sol(t_plot)[0] # requires `dense_output=True` in `solve_ivp`
x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, x_p0, y_p0) # pivot coordinates
x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l) # pendulum coordinates of pendulum (referenced to pivot point)
x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum) # pendulum coordinates in the lab frame

# animation flags/parameters
show_traj = True # True --> show trajectory; False --> do not show trajectory
speed_factor = 1 # >1 --> slow down animation; <1 --> speed up animation

# create_pendulum_animation
pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, show_traj=show_traj, speed_factor=speed_factor) # create animation object with trajectory shown
HTML(pendulum_animation.to_html5_video()) # display animation

##Analysis of first animation
With these conditions and this method of solving the equation, it appears that we have a good looking animation that describes the motion of a rigid pendulum based on my understanding of it.


##Analyzing the behavior as we vary the driving force
First we need to make a function that does the above steps for us given different driving force inputs. I would also like to add another input for the amount of cycles that it will animate.

In [ ]:
#graphing function
def graph_pendulum(omega_d, cycles):
  g = 1
  theta_0 = 0
  omega_0 = 1
  l=1
  ang_vel_0 = omega_d/l
  gamma=0.1

  #packing state variable
  y_0 = [theta_0, ang_vel_0]



  t_min = 0
  t_max = cycles * 2 * np.pi / omega_d
  t_span = [t_min, t_max] # time span for solve_ivp

  sol = solve_ivp(driven, t_span, y_0, args=(g, l, omega_d, gamma),
                    dense_output=True,
                    method='Radau'
                    )
  # animate motion
  # plot pendulum angle versus time
  # get coordinates for plotting
  n_points = 300 # number of frames
  t_plot = np.linspace(t_min, t_max, n_points)
  theta_plot = sol.sol(t_plot)[0] # requires `dense_output=True` in `solve_ivp`
  x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, x_p0, y_p0) # pivot coordinates
  x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l) # pendulum coordinates of pendulum (referenced to pivot point)
  x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum) # pendulum coordinates in the lab frame

  # animation flags/parameters
  show_traj = True # True --> show trajectory; False --> do not show trajectory
  speed_factor = omega_d # >1 --> slow down animation; <1 --> speed up animation

  # create_pendulum_animation
  pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, show_traj=show_traj, speed_factor=speed_factor) # create animation object with trajectory shown
  return HTML(pendulum_animation.to_html5_video()) # display animation

##Running this function for increasing values of driving force
Let's see what happens as the driving force gets very large

In [ ]:
display(graph_pendulum(500, 10))

In [ ]:
display(graph_pendulum(1000, 10))

In [ ]:
display(graph_pendulum(5000, 10))

In [ ]:
display(graph_pendulum(10000, 10))

#Conclusion

When the driving frequency is extremely large (about  times the natural frequency) and the damping ratio is , the pendulum bob remains steady at the center because its amplitude of oscillation is inversely proportional to the square of the driving frequency, becoming effectively zero due to inertia preventing it from following the rapid driving force.